# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment,Environment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-final-project-2")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-186279
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-186279


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [82]:
filepath="https://raw.githubusercontent.com/soorya-venkatesh/nd00333-capstone/master/starter_file/Telco_customer_churn.csv"
ds = TabularDatasetFactory.from_delimited_files(path = filepath)

df=ds.to_pandas_dataframe()
df.head(3)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,True,Mailed check,53.85,108.15,True,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,True,Electronic check,70.70,151.65,True,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,True,Electronic check,99.65,820.50,True,1,86,5372,Moved


In [88]:
df.describe

<bound method NDFrame.describe of       CustomerID  Count        Country       State          City  Zip Code  \
0     3668-QPYBK      1  United States  California   Los Angeles     90003   
1     9237-HQITU      1  United States  California   Los Angeles     90005   
2     9305-CDSKC      1  United States  California   Los Angeles     90006   
3     7892-POOKP      1  United States  California   Los Angeles     90010   
4     0280-XJGEX      1  United States  California   Los Angeles     90015   
...          ...    ...            ...         ...           ...       ...   
7038  2569-WGERO      1  United States  California       Landers     92285   
7039  6840-RESVB      1  United States  California      Adelanto     92301   
7040  2234-XADUH      1  United States  California         Amboy     92304   
7041  4801-JZAZL      1  United States  California  Angelus Oaks     92305   
7042  3186-AJIEK      1  United States  California  Apple Valley     92308   

                    Lat Long 

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# Choose a name for your CPU cluster
cpu_cluster_name = "compute-test"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.7
- scikit-learn
- pandas
- numpy
- pip:
  - azureml-defaults

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-186279', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-186279'), name=compute-test, id=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourceGroups/aml-quickstarts-186279/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-186279/computes/compute-test, type=AmlCompute, provisioning_state=Succeeded, location=southcentralus, tags={})

In [4]:
env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
import os
from azureml.train.hyperdrive import uniform, choice, randint




# Specify parameter sampler
ps = RandomParameterSampling( {
    'max_depth': choice(range (2, 5, 1)),
    'n_estimators': choice(range(50, 560, 100)),
    'min_samples_split': choice(2,3,5),
    'min_samples_leaf':choice(1,3,5)   
    }
)



# Specify a Policy
policy = BanditPolicy(slack_factor = 0.17, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# # Create a SKLearn estimator for use with train.py
# est = SKLearn(source_directory='./',  
#               compute_target=cpu_cluster,
#               entry_script='train.py', 
#               vm_size = 'STANDARD_D13_V2', 
#               vm_priority = 'lowpriority'
#             )

est = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target=cpu_cluster,
                            environment=env)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='AUC-ROC',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_concurrent_runs=8,
                                     max_total_runs=15)

In [7]:
#TODO: Submit your experiment
experiment = exp.submit(config=hyperdrive_config)


In [8]:
RunDetails(experiment).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
import joblib

best_run = experiment.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']


print('Experiment Run details:',best_run)
print('Best Accuracy after tuning tuning parameters:',best_run_metrics['AUC-ROC'])
print('Best hyper parameter:',parameter_values)

Experiment Run details: Run(Experiment: udacity-final-project-2,
Id: HD_fb1a187e-9d12-4f7f-bd58-8cc2843da549_4,
Type: azureml.scriptrun,
Status: Completed)
Best Accuracy after tuning tuning parameters: 0.8595998346637733
Best hyper parameter: ['--max_depth', '4', '--min_samples_leaf', '5', '--min_samples_split', '3', '--n_estimators', '50']


In [10]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-final-project-2,HD_fb1a187e-9d12-4f7f-bd58-8cc2843da549_4,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [28]:
#TODO: Save the best model
# saving the best model
best_model = best_run.register_model(model_name='best_model', model_path='outputs/model.joblib')
best_model.download(target_dir='out', exist_ok=True)


'out/model.joblib'

In [34]:
best_model.name,best_model.version

('best_model', 1)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [51]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice

inference_config = InferenceConfig(entry_script="score.py", environment = env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1,enable_app_insights = True)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


In [71]:
best_model

Model(workspace=Workspace.create(name='quick-starts-ws-186279', subscription_id='f9d5a085-54dc-4215-9ba6-dad5d86e60a0', resource_group='aml-quickstarts-186279'), name=best_model, id=best_model:1, version=1, tags={}, properties={})

TODO: In the cell below, send a request to the web service you deployed to test it.

In [53]:
service=Model.deploy(workspace=ws,
                    name="test-deploy",
                    models=[best_model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-17 12:41:46+00:00 Creating Container Registry if not exists..
2022-02-17 12:51:47+00:00 Registering the environment.
2022-02-17 12:51:47+00:00 Use the existing image.
2022-02-17 12:51:47+00:00 Generating deployment configuration..
2022-02-17 12:51:48+00:00 Submitting deployment to compute..
2022-02-17 12:51:54+00:00 Checking the status of deployment test-deploy..
2022-02-17 12:54:50+00:00 Checking the status of inference endpoint test-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [55]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://8d59dea4-8529-4968-8fae-b32e81b1e65c.southcentralus.azurecontainer.io/score

swagger URI: 
http://8d59dea4-8529-4968-8fae-b32e81b1e65c.southcentralus.azurecontainer.io/swagger.json

http://8d59dea4-8529-4968-8fae-b32e81b1e65c.southcentralus.azurecontainer.io/score
http://8d59dea4-8529-4968-8fae-b32e81b1e65c.southcentralus.azurecontainer.io/swagger.json


In [57]:
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

filepath="https://raw.githubusercontent.com/soorya-venkatesh/nd00333-capstone/master/starter_file/Telco_customer_churn.csv"
ds = TabularDatasetFactory.from_delimited_files(path = filepath)
#df = ds.to_pandas_dataframe()
x,y =clean_data(ds)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y)


In [76]:
req = x_test[:3].values.tolist()
req = [req[0][:],req[1][:],req[2][:]]

In [80]:
import requests
import json
scoring_uri='http://ef864eb6-2c46-4540-8f46-7e6cca62315b.southcentralus.azurecontainer.io/score'
data = json.dumps({"data":req})
headers = {'Content-Type':'application/json'}

response = requests.post(scoring_uri,data,headers=headers)
print(response.text)

[0, 0, 0]


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

